In [1]:
import os
import time
import numpy as np
import pandas as pd
import pickle as pkl
import pyomo.environ as pyomo
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

#Change this to point to your version of cbc or use another solver
solver_kwargs={'_name': 'cbc', 'executable': 'src/cbc'}

In [2]:
deep_reload(src)
'''
Solving the CHRP

reservoirs will be said to have around 10 hydro units of capacity
reservoir lower bounds will be around 5 hydro units
reservoir upper bounds will be around 9 hydro units
generator and pump operating ranges will be in the range of 1 to 2 hydro unit per hour
transmission cost to purchase will be around 1 cost unit per power unit
transmission cost to sell will be around -0.8 cost unit per power unit
rainfall will be around 0.1 hydro units per hour
power demand will be around 1 power units per hour
'''

n = 5 #reservoirs
m = 24 #hours

inputs={
    'n_r': n,
    'n_t': m,
    'c_tp': np.ones((m,)) * 1 + np.random.rand(m) * 0,
    'c_ts': -(np.ones((m,)) * .8 + np.random.rand(m) * 0),
    'y_e': np.ones((m,)) * 1 + np.random.rand(m) * .1,
    'y_f': np.ones((m, n)) * .1 + np.random.rand(m, n) * .01,
    'lb': np.ones((n,)) * 5 + np.random.rand(n) * 0,
    'ub': np.ones((n,)) * 9 + np.random.rand(n) * 0,
    'fs': np.ones((n,)) * 7 + np.random.rand(n) * 0,
    'is': np.ones((n,)) * 7 + np.random.rand(n) * 0,
    'f_g_min': np.ones((n,)) * 1 + np.random.rand(n) * 0,
    'f_g_max': np.ones((n,)) * 2 + np.random.rand(n) * 0,
    'f_p_min': np.ones((n,)) * 1 + np.random.rand(n) * 0,
    'f_p_max': np.ones((n,)) * 2 + np.random.rand(n) * 0,
    'tp_max': 10,
    'ts_max': 10,
}

t0 = time.time()
problem = src.optimization.CHRP(inputs)

t1 = time.time()
problem.Solve(solver_kwargs)

t2 = time.time()

print(problem.model.objective(), t1 - t0, t2 - t1)

12.380744275 0.012818336486816406 0.10673999786376953


In [3]:
deep_reload(src)
'''
Solving the S-CHRP

reservoirs will be said to have around 10 hydro units of capacity
reservoir lower bounds will be around 5 hydro units
reservoir upper bounds will be around 9 hydro units
generator and pump operating ranges will be in the range of 1 to 2 hydro unit per hour
transmission cost to purchase will be around 1 cost unit per power unit
transmission cost to sell will be around -0.8 cost unit per power unit
rainfall will be around 0.1 hydro units per hour
power demand will be around 1 power units per hour

In this scenario several discrete scenarios will be stochastically optimized for.
Stochastic exogenous inputs will be rainfall and demand power
'''

n = 5 #reservoirs
m = 24 #hours
s = 3 #scenarios

inputs={
    'n_r': n,
    'n_t': m,
    'n_s': s,
    'c_tp': np.ones((m, )) * 1 + np.random.rand(m) * 0,
    'c_ts': -(np.ones((m, )) * .8 + np.random.rand(m) * 0),
    'c_tpd': np.ones((m, )) * 2 + np.random.rand(m) * 0,
    'c_tsd': -(np.ones((m )) * .4 + np.random.rand(m) * 0),
    'y_e': np.ones((m, s)) * 1 + np.random.rand(m,s) * .1,
    'y_f': np.ones((m, n, s)) * .1 + np.random.rand(m,n,s) * .01,
    'lb': np.ones((n, )) * 5 + np.random.rand(n) * 0,
    'ub': np.ones((n, )) * 9 + np.random.rand(n) * 0,
    'fs': np.ones((n, )) * 7 + np.random.rand(n) * 0,
    'is': np.ones((n, )) * 7 + np.random.rand(n) * 0,
    'f_g_min': np.ones((n, )) * 1 + np.random.rand(n) * 0,
    'f_g_max': np.ones((n, )) * 2 + np.random.rand(n) * 0,
    'f_p_min': np.ones((n, )) * 1 + np.random.rand(n) * 0,
    'f_p_max': np.ones((n, )) * 2 + np.random.rand(n) * 0,
    'tp_max': 10,
    'ts_max': 10,
    'tpd_max': 3,
    'tsd_max': 3,
    'p_s': np.ones((s, )) / s
}

t0 = time.time()
problem = src.optimization.SCHRP(inputs)

t1 = time.time()
problem.Solve(solver_kwargs)

t2 = time.time()

print(problem.model.objective(), t1 - t0, t2 - t1)

12.574134499799998 0.029927968978881836 1.7912776470184326
